# GO enrichment tool
Copyright (c) 2017 Pieter Moris, based on code by Pieter Meysman

In [64]:
import pandas as pd
import argparse, re, os

In [96]:
def importGAF(path, geneSet):
    """
    Imports a GAF file (gene association format) and generates a dictionary mapping the gene uniprot AC to the GO ID.
    
    Information on the GAF 2.1 format can be found at 
        http://geneontology.org/page/go-annotation-file-gaf-format-21 
            
    Parameters
    ----------
    path : str
        The path to the file.
    geneSet : set
        A set containing the uniprot AC's of all the genes under consideration (background).

    Returns
    -------
    dict of str
        A dictionary mapping gene uniprot AC's to GO ID's.
        
        
    Possible improvements:
        Check for `is_obsolete` and `replaced_by`, although the replacement term should be in OBO file as an entry.
        
        Check for inclusion in provided geneset afterwards using:
            gafDict = {key: value for key, value in gafDict.items() if key in geneSet }
            
        To do: double dictionary? already filter based on gene list? what to do with NOT?
    """
    
    gafPath = os.path.abspath(path)
    
    if not geneSet:
        print('Empty gene set was provided during creation of GAF dictionary.')
        exit()
    
    with open(gafPath,'r') as gafFile:
        gafDict = {}
        for line in gafFile:
#             if not line.startswith('!'):                    # ignore comments in gaf file
            if line.startswith('UniProtKB'):
                splitLine = line.split('\t')                # split column-wise
                uniprotAC = splitLine[1]
                goTerm = splitLine[4]
                goQualifier = splitLine[3]
                if 'NOT' not in goQualifier:                # ignore annotations with "NOT"
                    if uniprotAC in geneSet:                # only keep genes present in background gene set
                        if uniprotAC not in gafDict:        # Create new key if AC does not already appear in dictionary
                            gafDict[uniprotAC] = {goTerm}   # make dictionary containing uniprot AC as key and set of GO's
                        else:
                            gafDict[uniprotAC].add(goTerm)
                            
    print('Retrieved',len(gafDict),'annotated (background filtered) uniprot AC\'s from',gafPath+'\n')
    
    if len(gafDict) != len(geneSet):
        print('WARNING!\nNot every uniprot AC that was provided in the background set was found in the GAF file:')
        [print(AC) for AC in geneSet if AC not in gafDict]
        print('WARNING!\n')
                            
    return gafDict

In [112]:
# https://stackoverflow.com/questions/1336791/dictionary-vs-object-which-is-more-efficient-and-why

'''
https://stackoverflow.com/questions/3489071/in-python-when-to-use-a-dictionary-list-or-set
When you want to store some values which you'll be iterating over, 
Python's list constructs are slightly faster. 
However, if you'll be storing (unique) values in order to check for their existence, 
then sets are significantly faster.
'''

class goTerm:
    """
    GO term object.
    
    Stores the ID, name and domain of the GO term and contains dictionaries for child and parent nodes.

    Attributes
    ----------
    id : str
        The identifier of the GO term.
    altid : str
        Optional tag for an alternative id.
    name : str
        The GO term name.
    namespace : str
        The domain of the GO term (Cellular Component, Molecular Function or Biological Process).
    parents : set of str
        The parent terms of the GO term, as indicated by the `is_a` relationship.
    childs : set of str
        The child terms of the GO term, derived from other GO terms after a complete OBO file is processed initially.

not necessary... Methods
    -------
    returnID
        Returns the ID of the GO term.
    gamma(n=1.0)
        Change the photo's gamma exposure.  
    """
    
    goCount = 0
    
    __slots__ = ('id', 'name', 'altid', 'namespace', 'childs', 'parents')
    
    def __init__(self, goID):
        self.id = goID
        self.altid = []
        self.name = ''
        self.namespace = ''
        self.childs = set()
        self.parents = set()
        
        goTerm.goCount += 1
    
    def returnID(self):
        return self.id
    

In [113]:
def importOBO(path):
    """
    Imports an OBO file and generates a dictionary containing an OBO object for each GO term.
    
    Parameters
    ----------
    path : str
        The path to the file.

    Returns
    -------
    dict of OBO objects
        Keys of the format `GO-0000001` mapping to OBO objects.
        
        
    Possible improvements:
        Check for `is_obsolete` and `replaced_by`, although the replacement term should be in OBO file as an entry.
    """
    
    GOdict = {}
    
    path = os.path.abspath(path)
    with open(path,'r') as oboFile:
        entryPattern = re.compile('^\[.+\]')             # find re pattern to match '[Entry]'
        validEntry = False
        
        for line in oboFile:
            
            if entryPattern.search(line):                # Only parse entries preceded by [Entry], not [Typedef]
                if 'Term' in line:
                    validEntry = True
                else:
                    validEntry = False
                    
            elif validEntry:                             # if [Entry] was encountered previously, parse annotation
                if line.startswith('id'):                # and hierarchy from subsequent lines
                    goID = line.split(': ')[1].rstrip()  # Store ID for lookup of other attributes in next lines
                    
                    if not goID in GOdict:               # check if id is already stored as a key in dictionary
                        GOdict[goID] = goTerm(goID)      # if not, create new goID object as the value for this key

                elif line.startswith('name'):
                    GOdict[goID].name = line.split(': ')[1].rstrip()
                elif line.startswith('namespace'):
                    GOdict[goID].namespace = line.split(': ')[1].rstrip()
                elif line.startswith('alt_id'):
                    GOdict[goID].altid.append(line.split(': ')[1].rstrip())
                elif line.startswith('is_a'):
                    GOdict[goID].parents.add(line.split()[1].rstrip())
    return GOdict



In [114]:
def importBackground(path):
    """
    Imports the background set of genes (uniprot AC).
    
    Parameters
    ----------
    path : str
        The path to the file.

    Returns
    -------
    set
        A set of background uniprot AC's.
        
    Notes: Gene lists should not contain a header. One gene per line.
    Possible improvement: check for file structure and allow headers, comma separated lists, etc.
    """
    with open(path,'r') as inGenes:
        backgroundSet = set([line.rstrip() for line in inGenes][1:])
    print('Retrieved',len(backgroundSet),'background uniprot AC\'s from',path)
    return backgroundSet

def importSubset(path):
    """
    Imports the gene subset of interest (uniprot AC).
    
    Parameters
    ----------
    path : str
        The path to the file.

    Returns
    -------
    set
        A subset of uniprot ACs of interest.
        
    Notes: Gene lists should not contain a header. One gene per line.
    """
    with open(path,'r') as inGenes:
        geneSubset = set([line.rstrip() for line in inGenes][1:])
    print('Retrieved',len(geneSubset),'subset uniprot AC\'s from',path)
    return geneSubset

In [128]:
# # Import gene list
# inputPath = os.path.abspath('../data/genelistinput.txt')
# with open(inputPath,'r') as inGenes:
#     geneSet = set([line.rstrip() for line in inGenes][1:])

backgroundPath = os.path.abspath('../data/background.txt')
subsetPath = os.path.abspath('../data/missing.txt')

# # Import .obo file
oboPath = os.path.abspath('../data/go_data/gotest.obo')
# with open(oboPath,'r') as oboFile:
#     obo = oboFile.read()

gafPath = os.path.abspath('../data/go_data/goa_human.gaf')

minGenes = 5
threshold = 0.1

background = importBackground(backgroundPath)
interest = importSubset(subsetPath)

gafDict = importGAF(gafPath, background)
gterms = importOBO(oboPath)

Retrieved 146 background uniprot AC's from /media/pieter/DATA/github/ebola-go/data/background.txt
Retrieved 26 subset uniprot AC's from /media/pieter/DATA/github/ebola-go/data/missing.txt
Retrieved 145 annotated (background filtered) uniprot AC's from /media/pieter/DATA/github/ebola-go/data/go_data/goa_human.gaf

WARNING!
Not every uniprot AC that was provided in the background set was found in the GAF file:
F8VVM2
WARNING!



In [129]:
print(gafDict['P84243'])
print(len(gafDict['P84243']))

{'GO:0008584', 'GO:0031492', 'GO:0006334', 'GO:0005654', 'GO:0045814', 'GO:0031508', 'GO:0000788', 'GO:0006336', 'GO:0048477', 'GO:0043234', 'GO:0046982', 'GO:0031509', 'GO:0005634', 'GO:0008283', 'GO:0000784', 'GO:0000786', 'GO:0090230', 'GO:0006997', 'GO:0035264', 'GO:0000980', 'GO:0005576', 'GO:0001649', 'GO:0007566', 'GO:0042393', 'GO:0070062', 'GO:1902340', 'GO:0007338', 'GO:0007596', 'GO:0000228', 'GO:0001740', 'GO:0000183', 'GO:0000979', 'GO:0007286', 'GO:0032200', 'GO:0042692', 'GO:0030307', 'GO:0031047', 'GO:0045815', 'GO:0005515', 'GO:0044267'}
40


In [130]:
# # Import .gaf file
# gafPath = os.path.abspath('../data/go_data/goa_human.gaf')
# with open(gafPath,'r') as gafFile:
#     gafDict = {}
#     for line in gafFile:
#         if not line.startswith('!'):                    # ignore comments in gaf file
#             splitLine = line.split('\t')                # split column-wise
#             uniprotAC = splitLine[1]
#             goTerm = splitLine[4]
#             goQualifier = splitLine[3]
#             if not 'NOT' in goQualifier:                # ignore annotations with "NOT"
#                 if uniprotAC in geneSet:                # only keep genes present in input gene set
#                     if not uniprotAC in gafDict:        # Create new key if AC does not already appear in dictionary
#                         gafDict[uniprotAC] = {goTerm}   # make dictionary containing uniprot AC as key and set of GO's
#                     else:
#                         gafDict[uniprotAC].add(goTerm)
# print(gafDict['O43175'])

In [131]:
# # Gaf pandas
# gafDf = pd.read_csv(gafPath, sep='\t', header=None, comment='!')
# gafDf = gafDf[gafDf[1].isin(geneSet)]    # only keep genes present in input gene set
# gafDf = gafDf[~gafDf[3].str.contains('NOT', na=False)]    # ignore annotations with "NOT"
# gafDf = gafDf.reset_index(drop=True)
# print(gafDf)

In [ ]:
print(gterms)
gterms['GO:0000001'].id
print(gterms['GO:0000001'].parents)
gterms['GO:0000001'].childs

In [124]:
def buildGOTree(GOdict):
    """
    Generates the entire GO tree by walking through the parent hierarchy.
    
    Parameters
    ----------
    GOdict : dict
        A dictionary of GO objects generated by importOBO().

    Returns
    -------
    dict of OBO objects
        Child and parent attributes are completed.
        Keys of the format `GO-0000001` mapping to OBO objects.
    """
    for goID in GOdict:
        
        # recursive call starts here
        
        for parent in GOdict[goID].parents:
            
            propagateTree(parent, goID, GOdict)
            
def propagateTree(parent, goID, GOdict):
    if goID not in GOdict[parent].child:
        GOdict[parent].child.add(goID)
        
    if GOdict[parent]:
        for parentsParent in GOdict[parent].parents: 
            propagateTree(parentsParent, goID, GOdict)
    else:
        return False

In [127]:
for goID in gterms:
    for parent in gterms[goID].parents:
        print(parent)
        
gterms['GO:0048308'].child

GO:0048308
GO:0048311
GO:0007005
GO:0008150


KeyError: 'GO:0048308'

In [126]:
buildGOTree(gterms)

KeyError: 'GO:0048308'

In [ ]:
from timeit import timeit
import re

def find(string, text):
    if string.find(text) > -1:
        pass

def re_find(string, text):
    if re.match(text, string):
        pass

def best_find(string, text):
    if text in string:
        pass

def third_find(string, text):
    if text.startswith(string):
        pass    

print(timeit("find(string, text)", "from __main__ import find; string='lookforme'; text='look'"))
print(timeit("re_find(string, text)", "from __main__ import re_find; string='lookforme'; text='look'")) 
print(timeit("best_find(string, text)", "from __main__ import best_find; string='lookforme'; text='look'")) 
print(timeit("third_find(string, text)", "from __main__ import third_find; string='lookforme'; text='look'")) 



In [ ]:
# https://techoverflow.net/blog/2013/11/18/a-geneontology-obo-v1.2-parser-in-python/
# Import .obo file
oboPath = os.path.abspath('../data/go_data/go.obo')
# with open(oboPath,'r') as oboFile:
#     for line in oboFile:


def processGOTerm(goTerm):
    """
    In an object representing a GO term, replace single-element lists with
    their only member.
    Returns the modified object as a dictionary.
    """
    ret = dict(goTerm) #Input is a defaultdict, might express unexpected behaviour
    for key, value in ret.iteritems():
        if len(value) == 1:
            ret[key] = value[0]
    return ret

def parseGOOBO(filename):
    """
    Parses a Gene Ontology dump in OBO v1.2 format.
    Yields each 
    Keyword arguments:
        filename: The filename to read
    """
    with open(filename, "r") as infile:
        currentGOTerm = None
        for line in infile:
            line = line.strip()
            if not line: continue #Skip empty
            if line == "[Term]":
                if currentGOTerm: yield processGOTerm(currentGOTerm)
                currentGOTerm = defaultdict(list)
            elif line == "[Typedef]":
                #Skip [Typedef sections]
                currentGOTerm = None
            else: #Not [Term]
                #Only process if we're inside a [Term] environment
                if currentGOTerm is None: continue
                key, sep, val = line.partition(":")
                currentGOTerm[key].append(val.strip())
        #Add last term
        if currentGOTerm is not None:
            yield processGOTerm(currentGOTerm)

In [ ]:
for goTerm in parseGOOBO(oboPath):
    termCounter += 1
